In [ ]:
#from os import chdir
#from google.colab import drive

#drive.mount("/content/drive", force_remount=True)
#chdir("/content/drive/MyDrive/Eliott/files/")

#!pip install tweet-preprocessor

Mounted at /content/drive


### Installation des dépendances

In [15]:
!pip install -r requirements.txt

In [2]:
# IMPORTS

import numpy as np
import pandas as pd
import preprocessing as pp
import json


from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline


%load_ext autoreload
%autoreload 2


In [3]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
#3 classes
X_train, y_train = pp.prepare_dataframe(file_name_train, False, lemmatising=False)
X_test, y_test = pp.prepare_dataframe(file_name_test, False, lemmatising=False)
#5 classes
X_train_5, y_train_5 = pp.prepare_dataframe(file_name_train, True, lemmatising=False)
X_test_5, y_test_5 = pp.prepare_dataframe(file_name_test, True, lemmatising=False)

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

- 35525 mots sans lemmatisation
- 30794 avec lemmatisation

# Recherche d'Hyperparamètres

In [4]:
pipeline_sgd = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])

pipeline_gb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', GradientBoostingClassifier()),
])

pipeline_rf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier()),
])

pipeline_lr = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression(max_iter=1000)),
])

pipeline_per = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', Perceptron()),
])

pipeline_svc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC(max_iter=10000)),
])


In [5]:
models_and_params = {
    "SGD" : { "model" : pipeline_sgd,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__alpha': np.linspace(1e-10, 1e-4, 40),  
                        }
    },
    "GB" : { "model" : pipeline_gb,
              "params" : {
                    "clf__learning_rate": [0.1, 0.2,0.4,0.8],
                    "clf__n_estimators":[1600]#,3200]
                        }
    },
    "RF" : { "model" : pipeline_rf,
              "params" : {
                    'clf__n_estimators': [100, 200],   
                    'clf__max_depth': [300,600,None],
                    'clf__min_samples_leaf': [1, 2, 3],   
                    'clf__min_samples_split': [16, 32, 64],
                    'clf__max_features': ['log2', 'sqrt'],
                    'clf__criterion': ['gini', 'entropy']
                        }
    },
    "LR" : { "model" : pipeline_lr,
              "params" : {
                    'clf__C': [20,10, 1],
                    'clf__tol': np.linspace(1e-12,1e-6,20)
                        }
    },
    "PER" : { "model" : pipeline_per,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__alpha': np.linspace(1e-8, 1e-4, 100),
                        }
    },
    "SVC" : { "model" : pipeline_svc,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__loss': ['hinge', 'squared_hinge'],
                    'clf__dual' : [False,True]
                        }
    }
    

}

In [6]:
def grid_Search(model_name,subset=-1):

    model = models_and_params[model_name]["model"]
    parameters = models_and_params[model_name]["params"]

    grid_clf = GridSearchCV(model, parameters,  scoring='accuracy', verbose=1 ,n_jobs=-1)
    
    if subset==-1:
        grid_clf.fit(X_train, y_train)
    else:
        grid_clf.fit(X_train[:subset], y_train[:subset])


    print("Best Score: ", grid_clf.best_score_)
    print("Best Params: ", grid_clf.best_params_)

    return grid_clf


### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [7]:
with open('data.json') as json_file:
    dico = json.load(json_file)

### SGD

In [ ]:
%%time
grid_sgd = grid_Search("SGD") 

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best Score:  0.8807008312810172
Best Params:  {'clf__alpha': 1.28206e-05, 'clf__penalty': 'l1'}
CPU times: user 29 s, sys: 4.25 s, total: 33.2 s
Wall time: 7min 37s


In [ ]:
dico['SGD'] = grid_sgd.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Gradient boosting

In [ ]:
%%time
grid_gb = grid_Search("GB") 

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Score:  0.8391769441865644
Best Params:  {'clf__learning_rate': 0.4, 'clf__n_estimators': 1600}
CPU times: user 33min 38s, sys: 7.7 s, total: 33min 46s
Wall time: 3h 40min 28s


In [ ]:
dico['GB'] = grid_gb.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Random Forest

In [ ]:
%%time
grid_rf = grid_Search("RF") 

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best Score:  0.7047644196092446
Best Params:  {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 32, 'clf__n_estimators': 200}
CPU times: user 2min 29s, sys: 14 s, total: 2min 43s
Wall time: 2h 30min 23s


In [ ]:
dico['RF'] = grid_rf.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)


### Logistic Regression

In [ ]:
%%time
grid_lr = grid_Search("LR") 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Score:  0.8230677619122002
Best Params:  {'clf__C': 10, 'clf__tol': 1e-12}
CPU times: user 1min 20s, sys: 2min 15s, total: 3min 35s
Wall time: 36min 42s


In [ ]:
dico['LR'] = grid_lr.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Perceptron

In [ ]:
%%time
grid_per = grid_Search("PER") 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Score:  0.818353925564533
Best Params:  {'clf__alpha': 3.04e-06, 'clf__penalty': 'l1'}
CPU times: user 1min 10s, sys: 7.51 s, total: 1min 17s
Wall time: 16min 16s


In [ ]:
dico['Perceptron'] = grid_per.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [ ]:
%%time
grid_svc = grid_Search("SVC") 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sk

Best Score:  0.868916566574562
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1'}
CPU times: user 11.5 s, sys: 524 ms, total: 12.1 s
Wall time: 1min 7s


In [ ]:
dico['SVC'] = grid_svc.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Charging the best parameters for training

In [8]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)


data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']

### Training on all training data and testing on test data

In [9]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  ")
    accuracy = []
    accuracy.append(accuracy_train)
    accuracy.append(accuracy_test)
    return accuracy

In [10]:
def testing_5_classes(model):
    predictions_train = model.predict(X_train_5)
    predictions_test = model.predict(X_test_5)
    accuracy_train = accuracy_score(y_train_5,predictions_train )
    accuracy_test = accuracy_score(y_test_5,predictions_test )
    
    print(f"train_accuracy for 5 classes : {accuracy_train} \ntest_accuracy for 5 classes: {accuracy_test}  ")
    accuracy = []
    accuracy.append(accuracy_train)
    accuracy.append(accuracy_test)
    return accuracy

In [11]:
accuracy_3_classes = []
accuracy_5_classes = []

### SGD

In [12]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**data_sgd,clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.0615789473684212e-05, n_jobs=-1,
                               penalty='l1'))])

**3 classes**

In [13]:
%%time

"""
Si on a fait la recherche avant, on prend le best modèle,

Sinon, on reconstruit le modèle avec les best paramètres et on le fit à toutes les données d'entrainement. 

"""

model_sgd_best.fit(X_train,y_train)


Wall time: 1.74 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.0615789473684212e-05, n_jobs=-1,
                               penalty='l1'))])

In [14]:
sgd_3_classes = testing(model_sgd_best)

train_accuracy : 0.9395728551643706 
test_accuracy : 0.8746708794102159  


In [15]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("sgd")
accuracy_3_classes_line.append(sgd_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [16]:
%%time

model_sgd_best.fit(X_train_5,y_train_5)

Wall time: 2.1 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.0615789473684212e-05, n_jobs=-1,
                               penalty='l1'))])

In [17]:
sgd_5_classes = testing_5_classes(model_sgd_best)

train_accuracy for 5 classes : 0.8270039118497461 
test_accuracy for 5 classes: 0.5992627698788836  


In [18]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("sgd")
accuracy_5_classes_line.append(sgd_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

### Random Forest

In [19]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**data_rf,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

**3 classes**

In [20]:
%%time

model_rf_best.fit(X_train,y_train)

Wall time: 8.61 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

In [21]:
rf_3_classes = testing(model_rf_best)

train_accuracy : 0.8651262239716209 
test_accuracy : 0.6924697209057399  


In [22]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("rf")
accuracy_3_classes_line.append(rf_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [23]:
%%time

model_rf_best.fit(X_train_5,y_train_5)

Wall time: 8.41 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

In [24]:
rf_5_classes = testing_5_classes(model_rf_best)

train_accuracy for 5 classes : 0.8289476881210973 
test_accuracy for 5 classes: 0.4673512374934176  


In [25]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("rf")
accuracy_5_classes_line.append(rf_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

### Gradient Boosting 

In [26]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**data_gb)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

**3 classes**

In [27]:
%%time
model_gb_best.fit(X_train,y_train)

Wall time: 13min 41s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

In [28]:
gb_3_classes = testing(model_gb_best)

train_accuracy : 0.9152513545690891 
test_accuracy : 0.8272775144813059  


In [29]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("gb")
accuracy_3_classes_line.append(gb_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [30]:
%%time
model_gb_best.fit(X_train_5,y_train_5)

Wall time: 32min 18s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

In [31]:
gb_5_classes = testing_5_classes(model_gb_best)

train_accuracy for 5 classes : 0.8844910950749568 
test_accuracy for 5 classes: 0.590047393364929  


In [32]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("gb")
accuracy_5_classes_line.append(gb_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

### Logistic Regression

In [33]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**data_lr,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

**3 classes**

In [34]:
%%time

model_lr_best.fit(X_train,y_train)

Wall time: 48.6 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

In [35]:
lr_3_classes = testing(model_lr_best)

train_accuracy : 0.9760429574555969 
test_accuracy : 0.8317535545023697  


In [36]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("lr")
accuracy_3_classes_line.append(lr_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [37]:
%%time

model_lr_best.fit(X_train_5,y_train_5)

Wall time: 1min 29s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

In [38]:
lr_5_classes = testing_5_classes(model_lr_best)

train_accuracy for 5 classes : 0.9501664358432345 
test_accuracy for 5 classes: 0.6040021063717746  


In [39]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("lr")
accuracy_5_classes_line.append(lr_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

### Perceptron

In [40]:
model_per_best = pipeline_per
model_per_best.set_params(**data_per,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

**3 classes**

In [41]:
%%time
model_per_best.fit(X_train,y_train)

Wall time: 3.16 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

In [42]:
per_3_classes = testing(model_per_best)

train_accuracy : 0.9071603858395899 
test_accuracy : 0.814902580305424  


In [43]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("per")
accuracy_3_classes_line.append(per_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [44]:
%%time
model_per_best.fit(X_train_5,y_train_5)

Wall time: 2.32 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

In [45]:
per_5_classes = testing_5_classes(model_per_best)

train_accuracy for 5 classes : 0.7100371747211895 
test_accuracy for 5 classes: 0.49921011058451814  


In [46]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("per")
accuracy_5_classes_line.append(per_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

### SVC

In [47]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**data_svc)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

**3 classes**

In [48]:
%%time

model_svc_best.fit(X_train,y_train)

Wall time: 16.5 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [49]:
svc_3_classes = testing(model_svc_best)

train_accuracy : 0.9475423378769103 
test_accuracy : 0.8670352817272249  


In [50]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("svc")
accuracy_3_classes_line.append(svc_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [51]:
%%time

model_svc_best.fit(X_train_5,y_train_5)

Wall time: 36.7 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [52]:
svc_5_classes = testing_5_classes(model_svc_best)

train_accuracy for 5 classes : 0.8556746118521759 
test_accuracy for 5 classes: 0.588204318062138  


In [53]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("svc")
accuracy_5_classes_line.append(svc_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

### Voting Classifier

In [54]:
from sklearn.ensemble import VotingClassifier


In [55]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


**3 classes**

In [56]:
%%time

eclf1.fit(X_train,y_train)

Wall time: 24min 28s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.0615789473684212e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(max_depth=600,
                                                                      max_features='sqrt',
                                                                      min_samples_leaf=2,
                                                                      min_samples_split=32,
                                                                  

In [57]:
eclf1_3_classes = testing(eclf1)

train_accuracy : 0.9582331073693418 
test_accuracy : 0.8686150605581885  


In [58]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("vc1")
accuracy_3_classes_line.append(eclf1_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

In [59]:
%%time

eclf2.fit(X_train,y_train)

Wall time: 34.3 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.0615789473684212e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                                  n_jobs=-1,
                                                                  tol=1e-08))])),
                             ('per',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),

In [60]:
eclf2_3_classes = testing(eclf2)

train_accuracy : 0.9572855164370581 
test_accuracy : 0.8636124275934702  


In [61]:
accuracy_3_classes_line = []
accuracy_3_classes_line.append("vc2")
accuracy_3_classes_line.append(eclf2_3_classes)
accuracy_3_classes.append(accuracy_3_classes_line)

**5 classes**

In [62]:
%%time

eclf1.fit(X_train_5,y_train_5)

Wall time: 24min 11s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.0615789473684212e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(max_depth=600,
                                                                      max_features='sqrt',
                                                                      min_samples_leaf=2,
                                                                      min_samples_split=32,
                                                                  

In [63]:
eclf1_5_classes = testing_5_classes(eclf1)

train_accuracy for 5 classes : 0.9143037636368054 
test_accuracy for 5 classes: 0.6142706687730385  


In [64]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("vc1")
accuracy_5_classes_line.append(eclf1_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

In [65]:
%%time

eclf2.fit(X_train_5,y_train_5)

Wall time: 54.2 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.0615789473684212e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                                  n_jobs=-1,
                                                                  tol=1e-08))])),
                             ('per',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),

In [66]:
eclf2_5_classes = testing_5_classes(eclf2)

train_accuracy for 5 classes : 0.8801175984644167 
test_accuracy for 5 classes: 0.5955766192733017  


In [67]:
accuracy_5_classes_line = []
accuracy_5_classes_line.append("vc2")
accuracy_5_classes_line.append(eclf2_5_classes)
accuracy_5_classes.append(accuracy_5_classes_line)

#### Comparaison du train et du test accuracy pour 3 classes et pour 5 classes pour les méthodes :
- SGD,
- Random Forest,
- Gradient Boosting,
- Logistic Regression,
- Perceptron,
- SVC et
- Voting Classifier de 2 versions :
    - SGD, Random Forest, Gradient Boosting, Logistic Regression, Perceptron, SVC
    - SGD, Logistic Regression, Perceptron, SVC

**3 classes**

In [144]:
list_inter_train = []
list_inter_test = []
for i in range(len(accuracy_3_classes)):
    list_inter_train.append(accuracy_3_classes[i][1][0])
    list_inter_test.append(accuracy_3_classes[i][1][1])

In [145]:
list_inter_train.sort()
list_inter_train.reverse()
list_inter_test.sort()
list_inter_test.reverse()

In [143]:
print("Le classement des methodes pour en fonction du meilleur \033[1mTrain accuracy\033[0m pour \033[1m3\033[0m classes est :")
print("1er Logistic Regression pour le train accuracy: ", list_inter_train[0])
print("2e Voting Classifier 1 pour le train accuracy:  ", list_inter_train[1])
print("3e Voting Classifier 2 pour le train accuracy:  ", list_inter_train[2])
print("4e SVC avec pour le train accuracy:             ", list_inter_train[3])
print("5e SGD avec pour le train accuracy:             ", list_inter_train[4])
print("6e Gradient Boosting pour le train accuracy:    ", list_inter_train[5])
print("7e Perceptron avec pour le train accuracy:      ", list_inter_train[6])
print("8e Random Forest pour le train accuracy:        ", list_inter_train[7])

Le classement des methodes pour en fonction du meilleur Train accuracy pour 3 classes est :
1er Logistic Regression pour le train accuracy:  0.9760429574555969
2e Voting Classifier 1 pour le train accuracy:   0.9582331073693418
3e Voting Classifier 2 pour le train accuracy:   0.9572855164370581
4e SVC avec pour le train accuracy:              0.9475423378769103
5e SGD avec pour le train accuracy:              0.9395728551643706
6e Gradient Boosting pour le train accuracy:     0.9152513545690891
7e Perceptron avec pour le train accuracy:       0.9071603858395899
8e Random Forest pour le train accuracy:         0.8651262239716209


In [148]:
print("Le classement des methodes pour en fonction du meilleur \033[1mTest accuracy\033[0m pour \033[1m3\033[0m classes est :")
print("1er SGD pour le train accuracy:                     ", list_inter_test[0])
print("2e Voting Classifier 1 pour le train accuracy:      ", list_inter_test[1])
print("3e SVC pour le train accuracy:                      ", list_inter_test[2])
print("4e Voting Classifier 2 avec pour le train accuracy: ", list_inter_test[3])
print("5e Logistic Regression avec pour le train accuracy: ", list_inter_test[4])
print("6e Gradient Boosting pour le train accuracy:        ", list_inter_test[5])
print("7e Perceptron avec pour le train accuracy:          ", list_inter_test[6])
print("8e Random Forest pour le train accuracy:            ", list_inter_test[7])

Le classement des methodes pour en fonction du meilleur Test accuracy pour 3 classes est :
1er SGD pour le train accuracy:                      0.8746708794102159
2e Voting Classifier 1 pour le train accuracy:       0.8686150605581885
3e SVC pour le train accuracy:                       0.8670352817272249
4e Voting Classifier 2 avec pour le train accuracy:  0.8636124275934702
5e Logistic Regression avec pour le train accuracy:  0.8317535545023697
6e Gradient Boosting pour le train accuracy:         0.8272775144813059
7e Perceptron avec pour le train accuracy:           0.814902580305424
8e Random Forest pour le train accuracy:             0.6924697209057399


**5 classes**

In [150]:
list_inter5_train = []
list_inter5_test = []
for i in range(len(accuracy_5_classes)):
    list_inter5_train.append(accuracy_5_classes[i][1][0])
    list_inter5_test.append(accuracy_5_classes[i][1][1])

In [151]:
list_inter5_train.sort()
list_inter5_train.reverse()
list_inter5_test.sort()
list_inter5_test.reverse()

In [157]:
print("Le classement des methodes pour en fonction du meilleur \033[1mTrain accuracy\033[0m pour \033[1m5\033[0m classes est :")
print("1er Logistic Regression avec pour accuracy: ", list_inter5_train[0])
print("2e Voting Classifier 2 avec pour accuracy:  ", list_inter5_train[1])
print("3e Gradient Boosting avec pour accuracy:    ", list_inter5_train[2])
print("4e Voting Classifier 1 avec pour accuracy:  ", list_inter5_train[3])
print("5e SVC avec pour accuracy:                  ", list_inter5_train[4])
print("6e Random Forest avec pour accuracy:        ", list_inter5_train[5])
print("7e SGD avec pour accuracy:                  ", list_inter5_train[6])
print("8e Perceptron avec pour accuracy:           ", list_inter5_train[7])

Le classement des methodes pour en fonction du meilleur Train accuracy pour 5 classes est :
1er Logistic Regression avec pour accuracy:  0.9501664358432345
2e Voting Classifier 2 avec pour accuracy:   0.9143037636368054
3e Gradient Boosting avec pour accuracy:     0.8844910950749568
4e Voting Classifier 1 avec pour accuracy:   0.8801175984644167
5e SVC avec pour accuracy:                   0.8556746118521759
6e Random Forest avec pour accuracy:         0.8289476881210973
7e SGD avec pour accuracy:                   0.8270039118497461
8e Perceptron avec pour accuracy:            0.7100371747211895


In [156]:
print("Le classement des methodes pour en fonction du meilleur \033[1mTest accuracy\033[0m pour \033[1m5\033[0m classes est :")
print("1er Voting Classifier 2 avec pour accuracy: ", list_inter5_test[0])
print("2e Logistic Regression avec pour accuracy:  ", list_inter5_test[1])
print("3e SGD avec pour accuracy:                  ", list_inter5_test[2])
print("4e Voting Classifier 1 avec pour accuracy:  ", list_inter5_test[3])
print("5e Gradient Boosting avec pour accuracy:    ", list_inter5_test[4])
print("6e SVC avec pour accuracy:                  ", list_inter5_test[5])
print("7e Perceptron avec pour accuracy:           ", list_inter5_test[6])
print("8e Random Forest avec pour accuracy:        ", list_inter5_test[7])

Le classement des methodes pour en fonction du meilleur Test accuracy pour 5 classes est :
1er Voting Classifier 2 avec pour accuracy:  0.6142706687730385
2e Logistic Regression avec pour accuracy:   0.6040021063717746
3e SGD avec pour accuracy:                   0.5992627698788836
4e Voting Classifier 1 avec pour accuracy:   0.5955766192733017
5e Gradient Boosting avec pour accuracy:     0.590047393364929
6e SVC avec pour accuracy:                   0.588204318062138
7e Perceptron avec pour accuracy:            0.49921011058451814
8e Random Forest avec pour accuracy:         0.4673512374934176


Ainsi on peut remarquer qu'en terme de **Train accuracy** le classement des 8 méthodes n'est pas le même pour 5 classes ou 3 classes. La 1ere méthode est la même mais pas le reste.

- 3 classes:
    - 1er Logistic Regression
    - 2e Voting Classifier 1
    - 3e Voting Classifier 2
    - 4e SVC
    - 5e SGD
    - 6e Gradient Boosting
    - 7e Perceptron
    - 8e Random Forest

- 5 classes :
    - 1er Logistic Regression
    - 2e Voting Classifier 2
    - 3e Gradient Boosting
    - 4e Voting Classifier 1
    - 5e SVC
    - 6e Random Forest
    - 7e SGD
    - 8e Perceptron

Ainsi on peut remarquer qu'en terme de **Test accuracy** le classement des 8 méthodes n'est pas le même pour 5 classes ou 3 classes. La 7e et 8e méthode sont les mêmes mais pas le reste.

- 3 classes :
    - 1er SGD
    - 2e Voting Classifier 1
    - 3e SVC
    - 4e Voting Classifier 2
    - 5e Logistic Regression
    - 6e Gradient Boosting
    - 7e Perceptron
    - 8e Random Forest

- 5 classes : 
    - 1er Voting Classifier 2
    - 2e Logistic Regression
    - 3e SGD
    - 4e Voting Classifier 1
    - 5e Gradient Boosting
    - 6e SVC
    - 7e Perceptron
    - 8e Random Forest

### Tests perso

In [ ]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

In [ ]:
predictions_out = eclf2.predict(tests_perso)


In [ ]:
predictions_out

array([-1,  1,  1])